<a href="https://colab.research.google.com/github/aliwagdy2580/NLP/blob/main/seq2seq_LSTM_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seq2seq models tutorial

In this tutorial we go through the details of building seq2seq models in Keras, and explore different models for different problem setups.

This tutorial includes:

- Vanilla seq2seq models for NMT.
- Effect of Embedding layer with zero masking.
- Adding attention + visualization: Luong vs. Bahdanau attention with LSTM.
- Char level models for OOV applications (spelling correction, chatbots).
- Issues with char models and possible fixes.

# Vanilla seq2seq

We will build a Neural Machine Translation (NMT) model based on the following paper: [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)

![Vanilla seq2seq](https://d3i71xaburhd42.cloudfront.net/673fa8ca55db79acdd88d50b465ec4580166fe09/2-Figure1-1.png)

In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

from keras.layers import Input, LSTM, Embedding, Dense, Bidirectional, Concatenate, Dot, Activation, TimeDistributed
from keras.models import Model


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data loading

In [3]:
!wget http://www.manythings.org/anki/fra-eng.zip
!unzip fra-eng.zip -d fra-eng


--2021-09-03 19:35:25--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.92.44, 172.67.186.54, 2606:4700:3030::6815:5c2c, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.92.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6451478 (6.2M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   6.15M  --.-KB/s    in 0.06s   

2021-09-03 19:35:25 (111 MB/s) - ‘fra-eng.zip’ saved [6451478/6451478]

Archive:  fra-eng.zip
  inflating: fra-eng/_about.txt      
  inflating: fra-eng/fra.txt         


In [4]:
data_path = 'fra-eng/fra.txt'
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
inputs = []
targets = []
num_samples = 10000  # Number of samples to train on.
for line in lines[: min(num_samples, len(lines) - 1)]:
  input, target, _ = line.split('\t')  
  inputs.append(input)
  targets.append(target)

In [5]:
lines=pd.DataFrame({'input':inputs,'target':targets})

In [6]:
num_samples = 10000
lines = lines[0:num_samples]


In [7]:
lines.shape

(10000, 2)

In [8]:
lines.head(10)

,input,target
0,Go.,Va !
1,Go.,Marche.
2,Go.,Bouge !
3,Hi.,Salut !
4,Hi.,Salut.
5,Run!,Cours !
6,Run!,Courez !
7,Run!,Prenez vos jambes à vos cous !
8,Run!,File !
9,Run!,Filez !


# Data preparation

In [9]:
def cleanup(lines):
  # Since we work on word level, if we normalize the text to lower case, this will reduce the vocabulary. It's easy to recover the case later. 
  lines.input=lines.input.apply(lambda x: x.lower())
  lines.target=lines.target.apply(lambda x: x.lower())

  # To help the model capture the word separations, mark the comma with special token:
  lines.input=lines.input.apply(lambda x:re.sub("'",'',x)).apply(lambda x: re.sub(",", ' COMMA', x))
  lines.target=lines.target.apply(lambda x:re.sub("'",'',x)).apply(lambda x: re.sub(",", ' COMMA', x))

  # Clean up punctuations and digits. Such special chars are common to both domains, and can just be copied with no error.
  exclude = set(string.punctuation)
  lines.input=lines.input.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  lines.target=lines.target.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

  remove_digits = str.maketrans('', '', digits)
  lines.input=lines.input.apply(lambda x: x.translate(remove_digits))
  lines.target=lines.target.apply(lambda x: x.translate(remove_digits))
  #return lines
    

In [10]:
st_tok='START_'
end_tok='_END'
def data_prep(lines):
  cleanup(lines)
  lines.target=lines.target.apply(lambda x: st_tok + ' ' + x + ' ' + end_tok)

To help the model identiy the start and end of sequence, we mark them by special tokens. This will help the decoding process later:

In [11]:
data_prep(lines)

In [12]:
lines.head(10)

,input,target
0,go,START_ va _END
1,go,START_ marche _END
2,go,START_ bouge _END
3,hi,START_ salut _END
4,hi,START_ salut _END
5,run,START_ cours _END
6,run,START_ courez _END
7,run,START_ prenez vos jambes à vos cous _END
8,run,START_ file _END
9,run,START_ filez _END


# Word level model (word2word)

In this section we process the text at the word level, both for `input` and `target` domains.

In [13]:
def tok_split_word2word(data):
  return data.split()

In [14]:
tok_split_fn = tok_split_word2word

In [15]:
def data_stats(lines, input_tok_split_fn, target_tok_split_fn):
  input_tokens=set()
  for line in lines.input:
      for tok in input_tok_split_fn(line):
          if tok not in input_tokens:
              input_tokens.add(tok)
      
  target_tokens=set()
  for line in lines.target:
      for tok in target_tok_split_fn(line):
          if tok not in target_tokens:
              target_tokens.add(tok)
  input_tokens = sorted(list(input_tokens))
  target_tokens = sorted(list(target_tokens))

  num_encoder_tokens = len(input_tokens)
  num_decoder_tokens = len(target_tokens)
  max_encoder_seq_length = np.max([len(input_tok_split_fn(l)) for l in lines.input])
  max_decoder_seq_length = np.max([len(target_tok_split_fn(l)) for l in lines.target])

  return input_tokens, target_tokens, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length



In [16]:
input_tokens,target_tokens,num_encoder_tokens,num_decoder_tokens,max_encoder_seq_length,max_decoder_seq_length=data_stats(lines,input_tok_split_fn=tok_split_fn,target_tok_split_fn=tok_split_fn)
print('Number of samples:', len(lines))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 2017
Number of unique output tokens: 4472
Max sequence length for inputs: 5
Max sequence length for outputs: 12


#Vocab

When building our vocab, we should keep a special entry for the PAD 

1.   List item
2.   List item

token, _always_ at the entry 0 of the vocab table. Of course this only happens when we pad with 0's. In general the padded token should be reserved, and NEVER used by any other token (otherwise we won't tell the difference between REAL 0 and PAD 0).

In [17]:
pad_tok = 'PAD'
sep_tok = ' '
special_tokens = [pad_tok, sep_tok, st_tok, end_tok] 
num_encoder_tokens += len(special_tokens)
num_decoder_tokens += len(special_tokens)

In [18]:

def vocab(input_tokens, target_tokens):
  
  input_token_index = {}
  target_token_index = {}
  for i,tok in enumerate(special_tokens):
    input_token_index[tok] = i
    target_token_index[tok] = i 

  offset = len(special_tokens)
  for i, tok in enumerate(input_tokens):
    input_token_index[tok] = i+offset

  for i, tok in enumerate(target_tokens):
    target_token_index[tok] = i+offset
   
  # Reverse-lookup token index to decode sequences back to something readable.
  reverse_input_tok_index = dict(
      (i, tok) for tok, i in input_token_index.items())
  reverse_target_tok_index = dict(
      (i, tok) for tok, i in target_token_index.items())
  return input_token_index, target_token_index, reverse_input_tok_index, reverse_target_tok_index

In [19]:
input_token_index, target_token_index, reverse_input_tok_index, reverse_target_tok_index = vocab(input_tokens, target_tokens)

# Vectorization

Also, when we vectorize, if we pad 0 (or any token), we need to set the corresponding target.

Below, we should set all `decoder_target_data` to the OHE corresponding the entry 0 (or whatever entry of PAD).

For example, if we pad with 0, and 0 is the first entry in vocab, then it's `decoder_target_data` must be [1,0,0,....0].
However, this is not needed if we use `mask_zero` in `Embedding`, or using a masking layer with LSTM, because those values wont be fed fwd anyway, and will not be considered in loss.

In [20]:
max_encoder_seq_length = 16
max_decoder_seq_length = 16

In [21]:
def init_model_inputs(lines, max_encoder_seq_length, max_decoder_seq_length, num_decoder_tokens):
  encoder_input_data = np.zeros(
      (len(lines.input), max_encoder_seq_length),
      dtype='float32')
  decoder_input_data = np.zeros(
      (len(lines.target), max_decoder_seq_length),
      dtype='float32')
  decoder_target_data = np.zeros(
      (len(lines.target), max_decoder_seq_length, num_decoder_tokens),
      dtype='float32')
  
  return encoder_input_data, decoder_input_data, decoder_target_data

In [22]:
  encoder_input_data, decoder_input_data, decoder_target_data = init_model_inputs(lines, max_encoder_seq_length, max_decoder_seq_length, num_decoder_tokens)


In [23]:
def vectorize(lines, max_encoder_seq_length, max_decoder_seq_length, num_decoder_tokens, input_tok_split_fn, target_tok_split_fn):
  encoder_input_data, decoder_input_data, decoder_target_data = init_model_inputs(lines, max_encoder_seq_length, max_decoder_seq_length, num_decoder_tokens)
  for i, (input_text, target_text) in enumerate(zip(lines.input, lines.target)):
      for t, tok in enumerate(input_tok_split_fn(input_text)):
          encoder_input_data[i, t] = input_token_index[tok]
      encoder_input_data[i, t+1:] = input_token_index[pad_tok]
      for t, tok in enumerate(target_tok_split_fn(target_text)):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_input_data[i, t] = target_token_index[tok]         
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_target_data[i, t - 1, target_token_index[tok]] = 1.
      decoder_input_data[i, t+1:] = target_token_index[pad_tok] 
      decoder_target_data[i, t:, target_token_index[pad_tok]] = 1.          
              
  return encoder_input_data, decoder_input_data, decoder_target_data              

In [24]:
encoder_input_data, decoder_input_data, decoder_target_data  = vectorize(lines, max_encoder_seq_length, max_decoder_seq_length, num_decoder_tokens, input_tok_split_fn=tok_split_fn, target_tok_split_fn=tok_split_fn)

# Model

In [25]:
emb_sz = 50

#### Encoder model

The LSTM is characterized by two internal tensors:
- cell state $C_t$: what to copy from past $C_{t-1}$ (after forgetting some info through sigmoid $f_t$) and what to take from the merge of last $h_{t-1}$ and current input $x_t$ 
- hidden state $h_t$: same as hidden state of normal RNNs. This affects the output of the LSTM Cell

We usually refer to the combination $[C_t, h_t]$ as the LSTM state at time step $t$.

So the inputs to LSTM are:
- $[C_{t-1}, h_{t-1}]$ = previous state
- $x_t$ = input tokens

And the outputs are:
- $[C_{t}, h_{t}]$


![LSTM Cell](https://www.researchgate.net/profile/Savvas_Varsamopoulos/publication/329362532/figure/fig5/AS:699592479870977@1543807253596/Structure-of-the-LSTM-cell-and-equations-that-describe-the-gates-of-an-LSTM-cell.jpg)

According to this nice [illustration](https://machinelearningmastery.com/return-sequences-and-return-states-for-lstms-in-keras/), when we feedfwd into LSTM, we can control the return from keras as follows:

- `return_state`: asks LSTM to return both states: $C_t$ and $h_t$. Three outputs are returned: $h_t$, $h_t$, $C_t$
- `return_sequence`: asks the unrollled LSTM to return its state $[C_t, h_t]$ for all $t \in [t_0,T]$. In this case 3 outputs are returned: [$h_{t_0}$, .., $h_T$], $h_T$, $C_T$

In the encoder model, we are interested in the last combined state [$h_t$, $C_t$]

#### Decoder model

In the decoder, we are interested in:
- intialize its hidden state from where the encoder ended `initial_state`=[$h_t$, $C_t$]
- the sequence outputs [$h_{t_0}$, .., $h_T$] returned thanks to `return_states`, which will be used to generate the output tokens through the final `softmax`.

The decoder is just another LSTM with the normal 2 inputs:
- $[C_{t-1}, h_{t-1}]$, where $[C_{0}, h_{0}]$ = encoder final state $[C_{T}, h_{T}]$
- $x_t$ = target tokens, shifted by one `start_tok` = `START_`, where $x_0$ = `START_` to kick-start the decoder input, coupled with the encoder final state.

Feeding the GT target tokens during training is called ___Teacher forcing___. This won't be straight forward using Keras, since it requires a dynamic graph to handle the _if_ condition of teacher forcing or free run mode (like in inference mode, taking the previous output). May be later using Eager execution.

During inference $x_t$ of the decoder, will just be the previous output $x_{t-1}$.

![enc-dec](https://miro.medium.com/max/3170/1*sO-SP58T4brE9EHazHSeGA.png)

Sometimes, it's helpful to switch between the GT target_tokens are inputs to the decoder during training, and sometimes take the previous output $x_{t-1}$ as will happen in the inference, so that the decoder is trained to recover from his mistakes.

In [26]:
def seq2seq(num_decoder_tokens, num_encoder_tokens, emb_sz, lstm_sz):
  #encoder
  encoder_inputs = Input(shape=(None,)) #create new layer
  en_x=  Embedding(num_encoder_tokens, emb_sz)(encoder_inputs) #embedding_vector  [vocab_sz, num_latent_factore]  create new layer
  encoder = LSTM(lstm_sz, return_state=True)  #create new layer
  encoder_outputs, state_h, state_c = encoder(en_x)
  # We discard `encoder_outputs` and only keep the states.
  encoder_states = [state_h, state_c]  #last state
  
  # Encoder model
  encoder_model = Model(encoder_inputs, encoder_states)
  
  #decoder5
  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = Input(shape=(None,)) #create new layer

  dex=  Embedding(num_decoder_tokens, emb_sz) # new embedding_vector  [vocab_sz, num_latent_factore]  create new layer

  final_dex= dex(decoder_inputs)


  decoder_lstm = LSTM(lstm_sz, return_sequences=True, return_state=True) #create new layer

  decoder_outputs, _, _ = decoder_lstm(final_dex,
                                      initial_state=encoder_states)

  decoder_dense = Dense(num_decoder_tokens, activation='softmax')

  decoder_outputs = decoder_dense(decoder_outputs)

  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


  
  # Decoder model: Re-build based on explicit state inputs. Needed for step-by-step inference:
  decoder_state_input_h = Input(shape=(lstm_sz,))
  decoder_state_input_c = Input(shape=(lstm_sz,))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]# last state

  decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex, initial_state=decoder_states_inputs)
  decoder_states2 = [state_h2, state_c2]
  decoder_outputs2 = decoder_dense(decoder_outputs2)
  decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs2] + decoder_states2)  

  return model, encoder_model, decoder_model


In [27]:
model, encoder_model, decoder_model = seq2seq(num_decoder_tokens, num_encoder_tokens, emb_sz, emb_sz)
print(model.summary())


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     101050      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     223800      input_2[0][0]                    
____________________________________________________________________________________________

#Fit the model

In [28]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=20,
          validation_split=0.05)

Epoch 1/20
149/149 [==============================] - 15s 43ms/step - loss: 3.0031 - acc: 0.7414 - val_loss: 1.7306 - val_acc: 0.7230
Epoch 2/20
149/149 [==============================] - 5s 36ms/step - loss: 1.3302 - acc: 0.8027 - val_loss: 1.5584 - val_acc: 0.7796
Epoch 3/20
149/149 [==============================] - 5s 36ms/step - loss: 1.1860 - acc: 0.8245 - val_loss: 1.4828 - val_acc: 0.7812
Epoch 4/20
149/149 [==============================] - 5s 36ms/step - loss: 1.1229 - acc: 0.8286 - val_loss: 1.4643 - val_acc: 0.7820
Epoch 5/20
149/149 [==============================] - 5s 36ms/step - loss: 1.0836 - acc: 0.8308 - val_loss: 1.4603 - val_acc: 0.7850
Epoch 6/20
149/149 [==============================] - 5s 36ms/step - loss: 1.0532 - acc: 0.8342 - val_loss: 1.4399 - val_acc: 0.7921
Epoch 7/20
149/149 [==============================] - 5s 36ms/step - loss: 1.0277 - acc: 0.8380 - val_loss: 1.4312 - val_acc: 0.7934
Epoch 8/20
149/149 [==============================] - 5s 36ms/step -

# Embedding + `mask_zero`

In [29]:
def seq2seq(num_decoder_tokens, num_encoder_tokens, emb_sz, lstm_sz):
  encoder_inputs = Input(shape=(None,))
  en_x=  Embedding(num_encoder_tokens, emb_sz, mask_zero=True)(encoder_inputs)
  encoder = LSTM(lstm_sz, return_state=True)
  encoder_outputs, state_h, state_c = encoder(en_x)
  # We discard `encoder_outputs` and only keep the states.
  encoder_states = [state_h, state_c]
  
  # Encoder model
  encoder_model = Model(encoder_inputs, encoder_states)
  
  
  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = Input(shape=(None,))

  dex=  Embedding(num_decoder_tokens, emb_sz, mask_zero=True)

  final_dex= dex(decoder_inputs)


  decoder_lstm = LSTM(lstm_sz, return_sequences=True, return_state=True)

  decoder_outputs, _, _ = decoder_lstm(final_dex,
                                      initial_state=encoder_states)

  decoder_dense = Dense(num_decoder_tokens, activation='softmax')

  decoder_outputs = decoder_dense(decoder_outputs)

  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


  
  # Decoder model: Re-build based on explicit state inputs. Needed for step-by-step inference:
  decoder_state_input_h = Input(shape=(lstm_sz,))
  decoder_state_input_c = Input(shape=(lstm_sz,))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

  decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex, initial_state=decoder_states_inputs)
  decoder_states2 = [state_h2, state_c2]
  decoder_outputs2 = decoder_dense(decoder_outputs2)
  decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs2] + decoder_states2)  

  return model, encoder_model, decoder_model
emb_sz = 256
model, encoder_model, decoder_model = seq2seq(num_decoder_tokens, num_encoder_tokens, emb_sz, emb_sz)
print(model.summary())
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=10,
          validation_split=0.2)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    517376      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    1145856     input_6[0][0]                    
____________________________________________________________________________________________

#### Function to generate sequences

In [32]:
def decode_sequence(input_seq, sep=' '):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq) # for init decoder input
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index[st_tok] #_START

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_tok = reverse_target_tok_index[sampled_token_index]
        decoded_sentence += sep + sampled_tok

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_tok == end_tok or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [35]:
for seq_index in range(20): #[14077,20122,40035,40064, 40056, 40068, 40090, 40095, 40100, 40119, 40131, 40136, 40150, 40153]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines.input[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 0    go
Name: input, dtype: object
Decoded sentence:  à la maison _END
-
Input sentence: 1    go
Name: input, dtype: object
Decoded sentence:  à la maison _END
-
Input sentence: 2    go
Name: input, dtype: object
Decoded sentence:  à la maison _END
-
Input sentence: 3    hi
Name: input, dtype: object
Decoded sentence:  à la maison _END
-
Input sentence: 4    hi
Name: input, dtype: object
Decoded sentence:  à la maison _END
-
Input sentence: 5    run
Name: input, dtype: object
Decoded sentence:  prenez la porte _END
-
Input sentence: 6    run
Name: input, dtype: object
Decoded sentence:  prenez la porte _END
-
Input sentence: 7    run
Name: input, dtype: object
Decoded sentence:  prenez la porte _END
-
Input sentence: 8    run
Name: input, dtype: object
Decoded sentence:  prenez la porte _END
-
Input sentence: 9    run
Name: input, dtype: object
Decoded sentence:  prenez la porte _END
-
Input sentence: 10    run
Name: input, dtype: object
Decoded sentence:  prenez la p